In [1]:
import pandas as pd
import re

In [2]:
df_stops = pd.read_csv("1960-stops-localised-final.csv")  

In [3]:
df_stops.drop(['Unnamed: 0'], axis=1, inplace=True)
df_stops['stop_name'] = df_stops['stop_name'].astype(str).str.strip()
df_stops['location'] = df_stops['location'].astype(str).str.strip()

In [4]:
df_stops.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1399 entries, 0 to 1398
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   stop_id           1399 non-null   int64 
 1   stop_description  1399 non-null   object
 2   type              1399 non-null   object
 3   in_lines          1399 non-null   object
 4   identifier        361 non-null    object
 5   location          1399 non-null   object
 6   stop_name         1399 non-null   object
dtypes: int64(1), object(6)
memory usage: 76.6+ KB


In [5]:
split_df = pd.concat([df_stops[col].astype(str).str.split(';', expand=True).stack().str.strip() for col in df_stops.columns], axis=1, keys=df_stops.columns)
df_stops = split_df.groupby(level=0).ffill().reset_index(drop=True)

In [6]:
df_stops = df_stops.fillna('').groupby(['stop_name', 'type', 'location']).agg({
    'in_lines': ', '.join,
    'identifier': ', '.join
}).reset_index()

# Reset the index
df_stops.reset_index(inplace=True)
df_stops.rename(columns={'index': 'stop_id'}, inplace=True)

df_stops.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1399 entries, 0 to 1398
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   stop_id     1399 non-null   int64 
 1   stop_name   1399 non-null   object
 2   type        1399 non-null   object
 3   location    1399 non-null   object
 4   in_lines    1399 non-null   object
 5   identifier  1399 non-null   object
dtypes: int64(1), object(5)
memory usage: 65.7+ KB


In [7]:
df_line = pd.read_csv("../initial/line_data_1960.csv")  

In [8]:
for index, row in df_line.iterrows():
    # Check if the regex pattern (lowercase 's' or 'x' at the end) is found in the "in_lines" column
    if re.search(r'[sx]$', row["line_name"].lower()):
        # Delete the section called for in the regex
        df_line.at[index, "line_name"] = re.sub(r'[sx]$', '', row["line_name"])


In [9]:
# Replace U+00a0 with regular whitespace in the entire DataFrame
df_line.replace('\u00a0', ' ', regex=True, inplace=True)

In [10]:
df_line.head(2)

,Stops,Frequency,Length (time),Length (km),line_name,type,east-west,year,start_stop,last_change
0,SchillingBrücke (Ostbahnhof) - U-Bhf. Strausbe...,20.0,33.0,7.7,1,strassenbahn,Ost,1960,NaN,NaN
1,Köpenicker Strasse Ecke Adalbertstrasse - Bhf....,20.0,23.0,5.4,11,strassenbahn,Ost,1960,NaN,NaN


In [11]:
# Assuming df is your DataFrame and 'stop_id' is the column you want to modify
df_stops['stop_id'] = '1960' + df_stops['stop_id'].astype(str)
df_stops.head(2)

,stop_id,stop_name,type,location,in_lines,identifier
0,19600,4. Ring,bus,"52.42250630689789, 13.2869490566549",A53,nan
1,19601,Adamstrasse,bus,"52.521029117218866, 13.19780699419913","A34,A35",nan


In [12]:
df_line['line_name'] = df_line['line_name'].astype(str).str.strip()
df_line['Stops'] = df_line['Stops'].astype(str).str.strip()
df_line['year'] = df_line['year'].astype(int)
df_line['Frequency'] = pd.to_numeric(df_line['Frequency'], errors='coerce').fillna(0).astype(int)
df_line['Length (time)'] = pd.to_numeric(df_line['Length (time)'], errors='coerce').fillna(0).astype(int)

In [13]:
# Define a function to remove leftover parentheses from a column
def remove_double_whitespace(text):
    text = text.replace("   ", " ")
    text = text.replace("  ", " ")
    text = text.replace("–", "-")
    text = text.replace(" - ", " - ")
    return text

# Apply the function to the 'line_stops' column
df_line['Stops'] = df_line['Stops'].apply(remove_double_whitespace)
df_line['Stops'] = df_line['Stops'].str.replace(r'\s*–\s*', ' - ', regex=True)

In [14]:
mask = df_line['Stops'].str.contains(" - ").fillna(True)
none_df = df_line[~mask]
none_df

,Stops,Frequency,Length (time),Length (km),line_name,type,east-west,year,start_stop,last_change


In [15]:
# Filter rows where 'line_stops' is not a string
non_string_rows = df_line[~df_line['Stops'].apply(lambda x: isinstance(x, str))]
non_string_rows

,Stops,Frequency,Length (time),Length (km),line_name,type,east-west,year,start_stop,last_change


In [16]:
### Creating related tables

In [17]:
def extract_first_start_stop(string):
    stations = string.split(" - ")
    first_station = stations[0]
    last_station = stations[-1]
    return f"{first_station}<> {last_station}"

In [18]:
def create_line_df(df):
    line_df = pd.DataFrame({
        'line_id': range(1, 1+len(df)),
        'year': df['year'],
        'line_name': df['line_name'],
        'type': df["type"],
        "start_stop": df['Stops'].apply(extract_first_start_stop),
        "Length (time)": df['Length (time)'],
        "Length (km)": df['Length (km)'],
        "east_west": df['east-west'],
        "Frequency": df['Frequency']
    })
    return line_df

In [19]:
line_df = create_line_df(df_line)

In [20]:
line_df.head(2)

,line_id,year,line_name,type,start_stop,Length (time),Length (km),east_west,Frequency
0,1,1960,1,strassenbahn,SchillingBrücke (Ostbahnhof)<> Am Kupfergraben,33,7.7,Ost,20
1,2,1960,11,strassenbahn,Köpenicker Strasse Ecke Adalbertstrasse<> Oran...,23,5.4,Ost,20


In [21]:
# Assuming df is your DataFrame and 'stop_id' is the column you want to modify
line_df['line_id'] = '1960' + line_df['line_id'].astype(str)
line_df.head(2)

,line_id,year,line_name,type,start_stop,Length (time),Length (km),east_west,Frequency
0,19601,1960,1,strassenbahn,SchillingBrücke (Ostbahnhof)<> Am Kupfergraben,33,7.7,Ost,20
1,19602,1960,11,strassenbahn,Köpenicker Strasse Ecke Adalbertstrasse<> Oran...,23,5.4,Ost,20


In [22]:
def create_line_stops_df(df):
    line_stops = df['Stops'].str.split(' - ', expand=True).stack().reset_index(level=1, drop=True).reset_index(name='stop_name')

    line_stops['stop_order'] = line_stops.groupby('index').cumcount()
    #index starts from 0 so it looks like 1 row is missing but this is not true

    # Clean the 'Stop Name' column by removing whitespace and non-breaking spaces
    line_stops['stop_name'] = line_stops['stop_name'].str.replace(u'\xa0', ' ').str.strip()

    # reset index so that it can be used for foreign key

    return line_stops

In [23]:
line_stops_df = create_line_stops_df(df_line)

In [24]:
for index, row in line_stops_df.iterrows():
    line_stops_df.at[index, "index"] = row["index"] + 1
line_stops_df.rename(columns={'index': 'line_id'}, inplace=True)

line_stops_df

,line_id,stop_name,stop_order
0,1,SchillingBrücke (Ostbahnhof),0
1,1,U-Bhf. Strausberger Platz,1
2,1,Stalinallee,2
3,1,Leninplatz,3
4,1,"Alexanderplats, Memhardstr.",4
...,...,...,...
2103,157,Südende,20
2104,157,Lankwitz,21
2105,157,Lichterfelde Ost,22
2106,157,Lichterfelde Süd,23


In [25]:
# Assuming df is your DataFrame and 'stop_id' is the column you want to modify
line_stops_df['line_id'] = '1960' + line_stops_df['line_id'].astype(str)
line_stops_df.head(2)

,line_id,stop_name,stop_order
0,19601,SchillingBrücke (Ostbahnhof),0
1,19601,U-Bhf. Strausberger Platz,1


In [26]:
# get a list of unique Line IDs in the line_stops DataFrame
line_ids = line_stops_df['line_id'].unique()

# loop through each unique Line ID
for line_id in line_ids:
    # get the first and last Stop Name for this Line ID
    first_stop_name = line_stops_df.loc[line_stops_df['line_id'] == line_id, 'stop_name'].iloc[0]
    last_stop_name = line_stops_df.loc[line_stops_df['line_id'] == line_id, 'stop_name'].iloc[-1]
    
    # get the corresponding Start-Stop value from line_df for this Line ID
    start_stop = line_df.loc[line_df['line_id'] == line_id, 'start_stop'].iloc[0]
    
    # check if the first and last Stop Names are contained in the Start-Stop value
    if first_stop_name not in start_stop or last_stop_name not in start_stop:
        print(f"Quality control check failed for line_id {line_id}.")

# I checked that this works using the following code:
# line_stops_df.loc[0, "stop_name"] = "test"
# and got the right result: Quality control check failed for line_id 1001.


In [27]:
line_df.head(3)

,line_id,year,line_name,type,start_stop,Length (time),Length (km),east_west,Frequency
0,19601,1960,1,strassenbahn,SchillingBrücke (Ostbahnhof)<> Am Kupfergraben,33,7.7,Ost,20
1,19602,1960,11,strassenbahn,Köpenicker Strasse Ecke Adalbertstrasse<> Oran...,23,5.4,Ost,20
2,19603,1960,13,strassenbahn,"Klingenberg, Blockdammweg<> Oderberger Strasse",37,9.9,Ost,7


In [28]:
def add_type(line_stops, line_df):
    # Assuming line_id is the common column between line_stops and line_df
    merged_df = pd.merge(line_stops, line_df[['line_id', 'type', "line_name"]], on='line_id', how='left')
    
    # Rename the 'type' column from line_df to 'type_from_line_df' to avoid conflicts
    merged_df.rename(columns={'type': 'type'}, inplace=True)
    
    # Drop the 'type_from_line_df' column if it's not needed in the final result
    # merged_df.drop(columns=['type_from_line_df'], inplace=True)
    
    return merged_df

In [29]:
line_stops_df = add_type(line_stops_df, line_df)

In [30]:
def add_fk(line_stops_df, df_stops):
    # Create a new dataframe with the Stop Name and Stop ID columns
    stop_id_df = df_stops[['stop_name', 'stop_id', 'type', 'in_lines']]

    # Merge the line_stops_df and stop_id_df dataframes based on matching stop names and line_name condition
    line_stops_df = line_stops_df.merge(stop_id_df,
                                        left_on=['stop_name', 'type'],
                                        right_on=['stop_name', 'type'],
                                        how='left')

    # Filter the rows based on the condition that line_name is contained in in_lines
    line_stops_df = line_stops_df[line_stops_df.apply(lambda row: str(row['line_name']) in str(row['in_lines']), axis=1)]

    return line_stops_df

# Assuming line_stops_df and df_stops are your dataframes
# Replace 'stop_name', 'stop_id', 'type', and 'in_lines' with the actual column names you have

line_stops_df = add_fk(line_stops_df, df_stops)


In [31]:
# Calculate the difference between consecutive 'stop_order' values
line_stops_df['diff'] = line_stops_df['stop_order'].diff()

# Use the following line to drop rows where the 'diff' column is 0.0
line_stops_df = line_stops_df[line_stops_df['diff'] != 0.0]

# Identify faulty rows where the difference is not 1 digit behind
faulty_rows = line_stops_df[(line_stops_df['diff'] != 1) & (line_stops_df['stop_order'] != 0)]
faulty_rows

,line_id,stop_name,stop_order,type,line_name,stop_id,in_lines,diff
424,196032,Schlossstrasse,4,strassenbahn,73,19601062,"74,78,44,73,77",2.0
440,196033,Schlossstrasse,4,strassenbahn,74,19601062,"74,78,44,73,77",2.0
518,196038,Bhf. Botanischer Garten,10,strassenbahn,77,1960124,"78,77",2.0
534,196039,Bhf. Botanischer Garten,10,strassenbahn,78,1960124,"78,77",2.0
1137,196079,Alt-Kladow,13,bus,A34,196021,"A34,A35",2.0
1156,196080,Alt-Kladow,13,bus,A35,196021,"A34,A35",2.0


In [32]:
faulty_rows.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6 entries, 424 to 1156
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   line_id     6 non-null      object 
 1   stop_name   6 non-null      object 
 2   stop_order  6 non-null      int64  
 3   type        6 non-null      object 
 4   line_name   6 non-null      object 
 5   stop_id     6 non-null      object 
 6   in_lines    6 non-null      object 
 7   diff        6 non-null      float64
dtypes: float64(1), int64(1), object(6)
memory usage: 432.0+ bytes


In [33]:
# convert 'Stop ID' column to numeric values, coercing errors to NaN
line_stops_df['stop_id'] = pd.to_numeric(line_stops_df['stop_id'], errors='coerce')

# check if all values in 'Stop ID' column are numeric
if line_stops_df['stop_id'].notnull().all():
    print("All values in 'stop_id' column are numeric")
else:
    print("There are non-numeric values in 'stop_id' column")
    print(line_stops_df[line_stops_df['stop_id'].isnull()])


All values in 'stop_id' column are numeric


In [34]:
line_stops_df.drop(['line_name', "in_lines", "type", "diff"], axis=1, inplace=True)

In [36]:
line_stops_df.to_csv("line_stops_1960-final.csv")
line_df.to_csv("line_df_1960-final.csv")
df_stops.to_csv("stops_df_1960-final.csv")